# Autómatas: AFD, AFN y AFN-Lambda

In [47]:
# Import libraries.
import numbers
import math
import numpy as np
import matplotlib.pyplot as plt

#### 1. Clase Autómata

---


Lorem Ipsum es simplemente el texto de relleno de las imprentas y archivos de texto. Lorem Ipsum ha sido el texto de relleno estándar de las industrias desde el año 1500, cuando un impresor (N. del T. persona que se dedica a la imprenta) desconocido usó una galería de textos y los mezcló de tal manera que logró hacer un libro de textos especimen. 

<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/9/9d/DFAexample.svg/400px-DFAexample.svg.png" width = "200px">

In [293]:
class Automata:
    
    def __init__(self, language = [], initial_state = 0, states = [], valid_states = [], delta = []):
        
        self.initial_state = initial_state
        self.language = language
        self.valid_states = valid_states
        self.adjMatrix = self.generarMatrizAdyacencia(states, delta)
        self.type = self.determinarTipoAutomata()
    
    def generarMatrizAdyacencia(self, states, delta):
        
        # Create matrix object.
        adjMatrix = list()
        
        # Number of layers of the adyacency matrix (depth).
        n = len(states)
        for i in range(len(self.language)):
              adjMatrix.append(np.zeros((n, n)))
                
        # Delta codifies the edges between nodes.
        for edge in delta:
            
            # Example: edge = ["q0", "q1", "a"]
            # Get states number.
            first_state = states.index(edge[0])
            second_state = states.index(edge[1])
            
            # Get instruction.
            instruction = self.language.index(edge[2])
            
            # Add new edge.
            adjMatrix[instruction][first_state][second_state] = 1
            
        # Return adyacency matrix.
        return adjMatrix

    # Utility functions.
    def obtenerNumerosEstadosAccesibles(self, current_state, instruction):
        layer = self.adjMatrix[instruction]
        bridges = layer[current_state]
        numbers = list()
        for b in range(len(bridges)):
            if bridges[b] == 1: numbers.append(b)
        return numbers
    
    def corroborarAceptacion(self, acceptanceList):
        if 1 in acceptanceList: return 1
        else: return 0   
        
    def determinarTipoAutomata(self): 
        
        """
        Type      Code
        ---------------
        AFD       0
        AFN       1
        AFN-e     2
        """
        if "e" in self.language: return 2
        else: 
            deterministic = True
            for instruction in self.language: 
                layer = self.adjMatrix[self.language.index(instruction)]
                for row_index in range(len(layer)):
                    lista = self.obtenerNumerosEstadosAccesibles(
                        row_index, 
                        self.language.index(instruction)
                    )
                    if len(lista) > 1: 
                        return 1
                    if len(lista) == 0:
                        deterministic = False
        if deterministic: 
            return 0
        else: 
            raise Exception("Todas las transiciones tienen que estar definidas!")
            
    def _obtenerSecuenciaEstadoInstruccion(self, cadena, recorrido):
        secuencia = list()
        for i in range(len(recorrido)): 
            try:
                secuencia.append((recorrido[i], cadena[i]))
            except:
                secuencia.append((recorrido[i], ""))
        print(secuencia)
        return secuencia 

In [294]:
delta = [
          ["q0", "q0", "a"], 
          ["q0", "q1", "b"], 
          ["q1", "q1", "b"], 
          ["q1", "q0", "a"], 
        ]

autom = Automata(language = ["a", "b"], states = ["q0", "q1"], delta = delta)

#### 2. Autómatas Deterministas.

In [314]:
class AFD(Automata):
    def __init__(self, language, initial_state, states, valid_states, delta): 
        super().__init__(language, initial_state, states, valid_states, delta)
        if self.type != 0: raise Exception("Los argumentos ingresados, no corresponden a un autómata determinista.")
        
    def procesarCadena(self, cadena):
        print(self.__procesar(cadena)[1] == 1)
    
    def procesarCadenaConDetalles(self, cadena): 
        process = self.__procesar(cadena)
        print(process[1] == 1)
        print(process[0])
    
    def procesarListaCadenas(self, listaCadenas, nombreArchivo, imprimirPantalla): 
        
        try:
            file = open(nombreArchivo, "w+")
        except: 
            file = open("resultadosAFD.txt", )
            
        for cadena in listaCadenas: 
            process = self.__procesar(cadena)
            secuencia = self._obtenerSecuenciaEstadoInstruccion(
                cadena,    # cadena
                process[0] # recorrido
            )
            file.write(cadena + "\t")
            for pareja in secuencia:
                file.write(str(pareja) + "\t")
            if process[1] == 1: aceptado = "Si"
            else: aceptado = "No"
            file.write(aceptado + "\n")
        
        f.close()
        return "Archivo creado con éxito"

    def __procesar(self, cadena, current_state = 0):
        
        # If chain is not null.
        if cadena != "":
            
            # Get next state number given by instruction in the chain.
            newState = self.obtenerNumerosEstadosAccesibles(
                current_state, 
                self.language.index(cadena[0])   
            )
            #print("New State: ", newState)
            newState = newState[0]

            # Move to next state.
            memory = self.__procesar(cadena[1:], current_state = newState)
            memory[0] = str(current_state) + memory[0]
            return memory

        # If chain is null.
        else: 
            if current_state in self.valid_states:
                return [str(current_state), 1]    # Accepted.
            else:
                return [str(current_state), -1]   # Rejected.

In [315]:
language = ["a", "b"]
initial_state = 0
states = ["q0", "q1"]
delta = [
          ["q0", "q0", "a"], 
          ["q0", "q1", "b"], 
          ["q1", "q1", "b"], 
          ["q1", "q0", "a"], 
        ]
afd = AFD(language = language, 
          initial_state = initial_state, 
          valid_states = [0], 
          states = states, 
          delta = delta)

In [316]:
afd.procesarCadenaConDetalles("abaaaaabab")

False
00100000101


In [317]:
afd.procesarListaCadenas(["abaaaaabab"], "res.txt", True)

[('0', 'a'), ('0', 'b'), ('1', 'a'), ('0', 'a'), ('0', 'a'), ('0', 'a'), ('0', 'a'), ('0', 'b'), ('1', 'a'), ('0', 'b'), ('1', '')]


'Archivo creado con éxito'

In [249]:
len("abaaaaab")

8

In [250]:
len("0010000010")

10

#### 3. Autómatas No-Deterministas.

In [150]:
class AFN(Automata):
    def __init__(self, language, initial_state, states, valid_states, delta): 
        super().__init__(language, initial_state, states, valid_states, delta)
        if self.type != 1: raise Exception("Los argumentos ingresados, no corresponden a un autómata no determinista.")
        self.procesador = ProcesamientoCadenaAFD()
        
    def procesarCadena(cadena):
        return ""
    
    def procesarCadenaConDetalles(cadena): 
        return ""
    
    def computarTodosLosProcesamientos(cadena): 
        return ""
    
    def procesarListaCadenas(listaCadenas, nombreArchivo, imprimirPantalla): 
        return ""
    
    def procesar(estado_actual, cadena):
        
    
        print("Current State: ", current_state)
        
        # If chain is not null.
        if cadena != "":
            
            # Get next state number given by instruction in the chain.
            newState = self.obtenerNumerosEstadosAccesibles(
                current_state, 
                self.language.index(cadena[0])   
            )[0]
            print("New State: ", newState)

            # Move to next state.
            return self.procesar()

        
        # If chain is null.
        else: 
            print("Chain is null!")
            if current_state in self.valid_states:
                return 1    # Accepted.
            else:
                return -1   # Rejected.
    

#### 4. Autómatas con transiciones lambda.

In [44]:
class AFNLambda(Automata):
    def __init__(self, language, initial_state, states, valid_states, delta): 
        super().__init__(language, initial_state, states, valid_states, delta)
        if self.type != 2: raise Exception("Los argumentos ingresados, no corresponden a un autómata con transiciones lambda.")
        procesador = ProcesamientoCadenaAFNLambda()
        
    def calcularLambdaClausura(estado):
        return ""
    
    def calcularLambdaClausuras(lista_estados):
        return ""
    
    def procesarCadena(cadena):
        return ""
    
    def procesarCadenaConDetalles(cadena): 
        return ""
    
    def computarTodosLosProcesamientos(cadena): 
        return ""
    
    def procesarListaCadenas(listaCadenas, nombreArchivo, imprimirPantalla): 
        return ""
    
    def procesar(cadena):
        return ""

#### 5. Clase Prueba.

In [45]:
class ClasePrueba:
    
    def __init__(self):
        return ""
    
    """
    Invoca a los otros para que puedan ser comentados fácilmente y poder 
    escoger cuál se va a probar.
    """
    def main():
        return ""
    
    """
    Crear autómatas AFD, procesar cadenas con y sin detalles, procesar listas
    de cadenas, generar archivos.
    """
    def probarAFD(self):
        return ""
    
    """
    Crear autómatas AFN, procesar cadenas mostrando solo un procesamiento de
    aceptación, procesar cadenas mostrando toos los porcesamientos posibles, 
    consultar los procesamientos de aceptación, abortados y de rechazo.
    """
    def probarAFN():
        return ""
    
    """
    Crear autómatas AFN-Lambda, calcular la Lambda-clausura de un estado, 
    calcular la Lambda-clausura de un conjunto de estados, procesar cadenas
    mostrando solo un procesamiento de aceptación, procesar cadenas mostrando
    todos los procesamientos posibles, consultar los procesaminetos de 
    aceptación, abortado y de rechazo, procesar listas de cadenas, generar 
    archivos.
    """
    def probarAFNLambda():
        return ""

In [ ]:
   # Get New States Numbers for lambda transition.
            #newStatesNumbersLambda = self.getStatesAccesibleNumbers(
            #    current_state, 
            #    self.language.index("e") 
            #)
            #print(current_state, "new states lamda: ", newStatesNumbersLambda)
            
            #if newStatesNumbersLambda != []:
                    #for newStateLambda in newStatesNumbersLambda:
                        #acceptance_list.append(self.processChain(newStateLambda, chain))
            

In [352]:
# Automata Class.
class Automata: 
    def __init__(self, language = [], initial_state = 0, states = [], valid_states = [], delta = []):
        
        #self.current_state = initial_state
        self.language = language
        self.valid_states = valid_states
        self.adjMatrix = self.generateAdyacencyMatrix(states, delta)
        
    def generateAdyacencyMatrix(self, states, delta):
        
        # Create matrix object.
        adjMatrix = list()
        
        # Number of layers of the adyacency matrix (depth).
        n = len(states)
        for i in range(len(self.language)):
              adjMatrix.append(np.zeros((n, n)))
                
        # Delta codifies the edges between nodes.
        for edge in delta:
            
            # Example: edge = ["q0", "q1", "a"]
            # Get states number.
            first_state = states.index(edge[0])
            second_state = states.index(edge[1])
            
            # Get instruction.
            instruction = self.language.index(edge[2])
            
            # Add new edge.
            adjMatrix[instruction][first_state][second_state] = 1
        
        # Return object.
        return adjMatrix
    
    # Process Chain.
    def processChain(self, current_state, chain):
        
        print("Current State: ", current_state)
        
        # If chain is not null.
        if chain != "":
            
            # Get New States Numbers for new instruction in the chain.
            newStatesNumbers = self.getStatesAccesibleNumbers(
                current_state, 
                self.language.index(chain[0])   
            )
            newStatesNumbersLambda = self.getStatesAccesibleNumbers(
                current_state, 
                self.language.index("e") 
            )
            print("New States Chain: ", newStatesNumbers)
            print("New States Lambda: ", newStatesNumbersLambda)
                    
            # If automata can move to new states.
            #print(newStatesNumbers)
            if newStatesNumbers != []:
                acceptance_list = list()
                for newState in newStatesNumbers:
                    acceptance_list.append(self.processChain(newState, chain[1:]))
                for newLambdaState in newStatesNumbersLambda: 
                    acceptance_list.append(self.processChain(newLambdaState, chain))
                return self.checkAcceptance(acceptance_list)
 
            # Aborted! Since there is no where to move.
            else: 
                return 0    # Aborted.       
        
        # If chain is null.
        else: 
            print("Chain is null!")
            if current_state in self.valid_states:
                return 1    # Accepted.
            else:
                return -1   # Rejected.
    
    # Get functions.
    def getAdyacencyMatrix(self, layer = 0):
        return (self.adjMatrix[layer])
    
    def getValidStates(self):
        return (self.valid_states)
    
    # Utility functions.
    def getStatesAccesibleNumbers(self, current_state, instruction):
        
        layer = self.adjMatrix[instruction]
        bridges = layer[current_state]
        numbers = list()
        for b in range(len(bridges)):
            if bridges[b] == 1: numbers.append(b)
        return numbers
    
    def checkAcceptance(self, acceptanceList):
        if 1 in acceptanceList: return 1
        else: return 0

In [353]:
"""
delta = [
          ["q0", "q0", "a"], 
          ["q0", "q1", "b"], 
          ["q1", "q1", "b"], 
          ["q1", "q0", "a"], 
        ]
delta = [
    ["q0", "q0", "a"], 
    ["q0", "q1", "a"], 
    ["q1", "q1", "b"], 
    ["q1", "q0", "a"], 
    ["q0", "q2", "e"], 
    ["q2", "q2", "a"],
    ["q2", "q1", "b"], 
]
"""
delta = [
    ["q0", "q0", "a"], 
    ["q0", "q1", "e"], 
]

autom = Automata(language = ["a", "b", "e"], 
                 states = ["q0", "q1"], 
                 delta = delta, 
                 valid_states = [1]
                )

In [354]:
autom.processChain(0, "a")

Current State:  0
New States Chain:  [0]
New States Lambda:  [1]
Current State:  0
Chain is null!
Current State:  1
New States Chain:  []
New States Lambda:  []


0

In [336]:
autom.getAdyacencyMatrix(2)

array([[0., 1.],
       [0., 0.]])

In [1]:
super()

RuntimeError: super(): no arguments

In [127]:
autom.getStatesAccesibleNumbers([0, 1, 0, 0, 1])

[1, 4]

In [119]:
autom.checkAcceptance([-1, 0, 0, 0])

0

In [48]:
lista = [0, 1, 0, 1, 0, 0, 1]

In [50]:
lista.index_all(1)

AttributeError: 'list' object has no attribute 'index_all'

In [106]:
chain = "a"

In [107]:
chain[1:]

''